In [1]:
import os
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import fuzz
import shortuuid
import xml.etree.ElementTree as ET
import re
import sys
import string
from setup import *

In [2]:
pd.set_option("display.max_colwidth", None)

# For Debugging:
#DATASETS = ["MPC", "SAv2"]

In [3]:
#Bring datasets to the same format (standardized)

df = pd.DataFrame(columns= [DATASET, PAIR_ID, ID1, ID2, TEXT1, TEXT2, PARAPHRASE] )

for dataset in DATASETS:
    path_to_dataset = os.path.join(DATASETS_FOLDER, dataset)
    print("Processing dataset: " + str(path_to_dataset))

    counter = 0

    df_tmp = pd.DataFrame(columns= [DATASET, PAIR_ID, ID1, ID2, TEXT1, TEXT2, PARAPHRASE] )

    if dataset == "MPC":
        dmop_path = os.path.join(path_to_dataset, "wikipedia_documents_train", "machined")      #read train data
        for file in tqdm(os.listdir(os.path.join(dmop_path, "og"))):
            with open(os.path.join(dmop_path, "og", file), encoding="utf8", mode = "r") as f1:
                with open(os.path.join(dmop_path, "mg", str(file.split("-")[0])+"-SPUN.txt"), encoding="utf8", mode = "r") as f2:
                    og_lines = f1.readlines()
                    og_lines = [line.rstrip() for line in og_lines]
                    og_lines = [l for l in og_lines if l != ""]
                    mg_lines = f2.readlines()
                    mg_lines = [line.rstrip() for line in mg_lines]
                    mg_lines = [l for l in mg_lines if l != ""]

                    if len(og_lines) != len(mg_lines):
                        print("ERROR")

                    for i, og_line in enumerate(og_lines):
                        counter = counter+1
                        if counter > 30:
                            break

                        if og_line != "\n":
                            df_tmp.loc[i] = [dataset, shortuuid.uuid()[:8], shortuuid.uuid()[:8], shortuuid.uuid()[:8], og_line, mg_lines[i], True]
        
        df = pd.concat([df, df_tmp], ignore_index = True)
        df_tmp = pd.DataFrame(columns= [DATASET, PAIR_ID, ID1, ID2, TEXT1, TEXT2, PARAPHRASE] )

        dmop_path = os.path.join(path_to_dataset, "wikipedia_documents_test", "machined")
        for file in tqdm(os.listdir(os.path.join(dmop_path, "og"))):        #read test data (combine as there is no ML process involved)
            with open(os.path.join(dmop_path, "og", file), encoding="utf8", mode = "r") as f1:
                with open(os.path.join(dmop_path, "mg", str(file.split("-")[0])+"-SPUN.txt"), encoding="utf8", mode = "r") as f2:
                    og_lines = f1.readlines()
                    og_lines = [line.rstrip() for line in og_lines]
                    og_lines = [l for l in og_lines if l != ""]
                    mg_lines = f2.readlines()
                    mg_lines = [line.rstrip() for line in mg_lines]
                    mg_lines = [l for l in mg_lines if l != ""]

                    for i, og_line in enumerate(og_lines):
                        counter = counter+1
                        if counter > 30:
                            break
                        if og_line != "\n":
                            df_tmp.loc[i] = [dataset, shortuuid.uuid()[:8], shortuuid.uuid()[:8], shortuuid.uuid()[:8], og_line, mg_lines[i], True]
    
    if dataset == "ETPC":
        with open(os.path.join(path_to_dataset, "text_pairs.xml"), encoding='utf-8', mode = "r") as file:
            tree = ET.parse(file)
            root = tree.getroot()
            for i, elem in enumerate(tqdm(root)):
                counter = counter+1
                if counter > 30:
                    break
                df_tmp.loc[i] = [dataset, shortuuid.uuid()[:8], elem[1].text, elem[2].text, elem[3].text, elem[4].text, bool(int(elem[8].text))]
    
    if dataset == "SAv2":
        asv2_path = os.path.join(path_to_dataset)      #read train data
        with open(os.path.join(asv2_path, "normal.aligned"), encoding="utf8", mode = "r") as f1:
            with open(os.path.join(asv2_path, "simple.aligned"), encoding="utf8", mode = "r") as f2:
                og_lines = f1.readlines()
                og_lines = [line.rstrip() for line in og_lines]
                og_lines = [l for l in og_lines if l != ""]
                mg_lines = f2.readlines()
                mg_lines = [line.rstrip() for line in mg_lines]
                mg_lines = [l for l in mg_lines if l != ""]

                for i, og_line in enumerate(tqdm(og_lines)):
                    counter = counter+1
                    if counter > 30:
                        break
                    if og_line != "\n":
                        df_tmp.loc[i] = [
                            dataset, 
                            shortuuid.uuid()[:8],
                            og_line.split("\t")[0].translate(str.maketrans('', '', string.punctuation+" ")) + "_" + shortuuid.uuid()[:8], 
                            mg_lines[i].split("\t")[0].translate(str.maketrans('', '', string.punctuation+" ")) + "_" + shortuuid.uuid()[:8], 
                            og_line.split("\t")[2], 
                            mg_lines[i].split("\t")[2], 
                            True
                        ]

    if dataset == "TURL":
        turl_path = os.path.join(path_to_dataset)      #read train data
        with open(os.path.join(turl_path, "Twitter_URL_Corpus_test.txt"), encoding="utf8", mode = "r") as f1:
            with open(os.path.join(turl_path, "Twitter_URL_Corpus_train.txt"), encoding="utf8", mode = "r") as f2:
                test_lines = f1.readlines()
                test_lines = [line.rstrip() for line in test_lines]
                test_lines = [l for l in test_lines if l != ""]
                train_lines = f2.readlines()
                train_lines = [line.rstrip() for line in train_lines]
                train_lines = [l for l in train_lines if l != ""]
                lines = test_lines + train_lines

                for i, line in enumerate(tqdm(lines)):
                    counter = counter+1
                    if counter > 30:
                        break
                    if line != "\n":
                        # based on the datasets paper, we value a phrase as paraphrase when >=4 out of 6 amazon workers marked it a such
                        is_paraphrase = int(line.split("\t")[2][1]) >= 4
                        df_tmp.loc[i] = [
                            dataset, 
                            shortuuid.uuid()[:8],
                            shortuuid.uuid()[:8], 
                            shortuuid.uuid()[:8], 
                            line.split("\t")[0], 
                            line.split("\t")[1], 
                            is_paraphrase
                        ]

    df = pd.concat([df, df_tmp], ignore_index = True)   #concat the lastly processed dataset to the combined dataset

Processing dataset: datasets\MPC


100%|██████████| 1990/1990 [00:00<00:00, 2515.24it/s]


Processing dataset: datasets\ETPC


  1%|          | 30/5801 [00:00<00:09, 638.13it/s]


Processing dataset: datasets\SAv2


  0%|          | 30/167689 [00:00<04:11, 666.52it/s]


Processing dataset: datasets\TURL


  0%|          | 30/56787 [00:00<01:27, 652.02it/s]


In [4]:
df.head(70000)

,dataset,pair_id,id_1,id_2,text_1,text_2,is_paraphrase
0,MPC,QG8smSKE,gbuGDKur,eJ9zMm6v,Uriel Sebree,Uriel Sebree,True
1,MPC,KVJu2kma,4MJMump6,2UnbDwSm,"Uriel Sebree (February 20, 1848 – August 6, 1922) was a career officer in the United States Navy. He entered the Naval Academy during the Civil War and served until 1910, retiring as a rear admiral. He is best remembered for his two expeditions into the Arctic and for serving as acting governor of American Samoa. He was also commander-in-chief of the Pacific Fleet.","Uriel Sebree (FebruaryÂ 20, 1848Â â AugustÂ 6, 1922) was a lifelong officer in the United States Navy. He entered the Naval Academy amid the Civil War and served until 1910, resigning as a back naval commander. He is best associated with his two endeavors into the Arctic and for filling in as acting legislative head of American Samoa. He was additionally president of the Pacific Fleet.",True
2,MPC,3U7CRXFC,hNBWbbMn,Q8PnnCBV,"After graduating from the U.S. Naval Academy in 1867, Sebree was posted to a number of vessels before being assigned to a rescue mission to find the remaining crew of the missing ""Polaris"" in the Navy's first mission to the Arctic. This attempt was only a partial success—the ""Polaris"" crew was rescued by a British ship rather than the US Navy—but this led to Sebree's selection eleven years later for a second expedition to the Arctic. That mission to rescue Adolphus Greely and the survivors of the Lady Franklin Bay expedition was a success. Sebree was subsequently appointed as the second acting governor of American Samoa. He served in this position for only a year before returning to the United States. In 1907, he was promoted to rear admiral and given command of the Pathfinder Expedition around the South American coast before being appointed commander of the 2nd Division of the Pacific Fleet and then commander-in-chief of the entire fleet. He retired in 1910 and died in Coronado, California, in 1922. Two geographical features in Alaska—Sebree Peak and Sebree Island—are named for Admiral Sebree.","In the wake of moving on from the U.S. Maritime Academy in 1867, Sebree was presented on various vessels before being appointed to a salvage mission to locate the rest of the team of the missing ""Polaris"" in the Navy's first mission to the Arctic. This endeavor was just a fractional achievementâthe ""Polaris"" group was saved by a British ship as opposed to the US Navyâyet this prompted Sebree's choice eleven years after the fact for a second campaign to the Arctic. That mission to safeguard Adolphus Greely and the overcomers of the Lady Franklin Bay endeavor was a triumph. Sebree was in this manner designated as the second acting legislative leader of American Samoa. He served in this situation for just a year prior coming back to the United States. In 1907, he was elevated to raise naval commander and given direction of the Pathfinder Expedition around the South American coast before being named authority of the second Division of the Pacific Fleet and afterward president of the whole armada. He resigned in 1910 and passed on in Coronado, California, in 1922. Two topographical highlights in AlaskaâSebree Peak and Sebree Islandâare named for Admiral Sebree.",True
3,MPC,cQ9Z8y56,Bdf6Z5tJ,3krDk95S,"""Super Mario 64"" is a 3D platformer in which the player controls Mario through several courses. Each course is an enclosed world in which the player is free to wander in all directions and discover the environment without time limits. The worlds are filled with enemies that attack Mario, as well as friendly creatures that provide assistance, offer information, or ask a favor (such as peaceful pink Bob-omb Buddies). The player gathers stars in each course; some stars only appear after completing certain tasks, often hinted at by the name of the course. These challenges include defeating a boss, solving puzzles, racing an opponent, and gathering coins. As more stars are collected, more areas 

In [5]:
#Output data to json format
df.to_json(os.path.join(OUT_DIR, "true_data.json"), orient = "index", index = True, indent = 4)
